In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
price = pd.read_csv('some_file_name',index_col=0)#'all_xl.csv'
price = pd.pivot_table(price,'close','date','code').ffill()
price.index = pd.to_datetime(price.index)
price = price[['each of the asset']]
ret = price.pct_change()
price

In [ ]:
name_list = ['what you call it' ]

excel_list = ['filename.xlsx']



In [ ]:
def plot_stack(df, sorted_index=None, title='', figsize=(16,9), benchmark=None):
    
    if sorted_index is None:
        sorted_index = df.sum().sort_values().index
    
    cm = plt.get_cmap('tab20')
    NUM_COLORS = len(sorted_index)
    color = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]
    
    
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)

    ax.set_prop_cycle(color=color[::-1])
    df1 = df.reindex(sorted_index[::-1], axis=1).applymap(lambda x: x if x>=0 else np.nan).fillna(0)
    df1.plot.area(ax=ax, linewidth=0)


    ax.set_prop_cycle(color=color)
    df0 = df.reindex(sorted_index, axis=1).applymap(lambda x: x if x<0 else np.nan)
    df0 = df0.rename(columns=lambda x: '_' + x)
    df0.plot.area(ax=ax, linewidth=0)


    pad = (df1.sum(axis=1).max() - df0.sum(axis=1).min())/20
    ax.set_ylim([df0.sum(axis=1).min()-pad, df1.sum(axis=1).max()+pad])
    
    
    df.sum(1).plot(ax=ax, style='k', linewidth=0.5, label='net')
    try:
        benchmark['TLT'].plot(ax=ax, style='k--', linewidth=0.5, label='TLT')
    except:
        print('benchmark not valid')
    
    ax.set_title(title)
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
def plot_all(excel_file_name, ret, ini='2007', start='2008', end='2021/12', name=''):
    
    result = {}
    
    kpi = pd.read_excel(excel_file_name, sheet_name='KPI', index_col=0)
    ykpi = pd.read_excel(excel_file_name, sheet_name='KPI(Annually)', index_col=0)
    mkpi = pd.read_excel(excel_file_name, sheet_name='KPI(Monthly)', index_col=0)
    
    allo = pd.read_excel(excel_file_name, sheet_name='History', index_col=0)
    
    allo = allo.iloc[1:]
    allo.index = pd.to_datetime(allo.index)
    try:
        allo = allo.drop(['cash','debit','aum','lev_cost','perf','commissions','asset','asset_gross'], 1)
    except:
        pass
    try:
        balance_ini = allo[ini].Balance.iloc[-1]
    except:
        balance_ini = 1e9
    
    allo = allo[start:end]
    
    balance_open = allo.Balance.shift().fillna(balance_ini)
    
    asset_value_close = allo.drop(['Balance'], 1)
    if int(end[:4]) < 2016:
        asset_value_close = asset_value_close.drop(['FSP','LSP','OSP','PC0',], 1)
        ret = ret.drop(['FSP','LSP','OSP','PC0',], 1)
    
    idx_allo = asset_value_close.index
    idx_ret = ret.index

    idx_common = list(set(idx_allo) & set(idx_ret))
    idx_common.sort()

    asset_value_close = asset_value_close.reindex(idx_common)
    ret_ = ret.reindex(idx_common)

    asset_value_open = asset_value_close/(1+ret_)
    asset_value_diff = asset_value_close - asset_value_open

    
    net_cum_ret = (1+ret_).cumprod()-1
    net_ret = (1+ret_).prod()-1
    net_ret.sort_values()
    
    
    net_aml_ret = asset_value_diff/balance_ini
    net_aml_cum_ret = net_aml_ret.cumsum()
    net_aml_total_ret = net_aml_ret.sum()
    net_aml_total_ret.sort_values()
    
    lev = pd.concat([(asset_value_open[a]/balance_open) for a in ret.columns], 1, keys=ret.columns)
    
    weighted_aml_ret = net_aml_ret / lev.mean()
    weighted_aml_cum_ret = weighted_aml_ret.cumsum()
    weighted_aml_total_ret = weighted_aml_ret.sum()
    weighted_aml_total_ret.sort_values()

    
    change_comp = pd.concat([net_ret, net_aml_total_ret, weighted_aml_total_ret],1,
          keys=['net_ret', 'net_aml_ret', 'weighted_aml_ret']).sort_index()
    title = '{} to {}'.format(start, end)
    change_comp.sort_values('net_aml_ret').plot.bar(figsize=(16,9), title=name+' '+title)
    
    
    daily_df_dict = {}
    for a in ret.columns:
        daily_df_dict[a] = pd.concat([net_cum_ret[a], net_aml_cum_ret[a], weighted_aml_cum_ret[a]],1,
              keys=['net_cum_ret', 'net_aml_cum_ret', 'weighted_aml_cum_ret'])
        ax = daily_df_dict[a][['net_cum_ret', 'weighted_aml_cum_ret']].plot(figsize=(12,6), title=a)
        ax.set_ylabel("cumulative return")
        ax.legend(loc='center right', bbox_to_anchor=(0, 1))
    
        twinx = ax.twinx()
        lev[a].plot(ax=twinx, style='r--', label='lev', alpha=0.3)
        twinx.set_ylabel("leverage")
        twinx.legend(loc='center left', bbox_to_anchor=(1, 1))
    
    # plot aml_cum_ret (stacked)
    
    net_aml_cum_ret_df = pd.DataFrame({a:daily_df_dict[a]['net_aml_cum_ret'] for a in ret.columns}).fillna(0)
    net_aml_cum_ret_df.plot(figsize=(16,9), title=name+' cumulative aml return')
    plot_stack(net_aml_cum_ret_df, title=name+' cumulative aml return (stacked)', benchmark=net_cum_ret)

    result['balance'] = allo.Balance
    result['kpi'] = kpi
    result['ykpi'] = ykpi
    result['mkpi'] = mkpi
    result['lev'] = lev
    result['net_aml_cum_ret_df'] = net_aml_cum_ret_df
    
    return result


In [ ]:
for n in excel_list:
    print(n)

In [ ]:
result_all = {}
for n, e in zip(name_list, excel_list):
    #result_all[n] = plot_all(e, ret, name=n)
    result_all[n] = plot_all(e, ret, ini='2007', start='2008', name=n)



In [ ]:
kpi_all = pd.concat([result_all[n]['kpi'] for n in name_list], 1, keys=name_list)
kpi_all

In [ ]:
for n in name_list:
    print(n)

In [ ]:
pd.concat([result_all[n]['ykpi'] for n in name_list], 1, keys=name_list).columns